In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
from sqlalchemy import create_engine

import os
import csv
from datetime import datetime

In [2]:
loc = '/data/SO_predict_DATA'
db_name = '/train_no_dup.db'

In [3]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(loc+db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
 
    return None

In [11]:
con = create_connection(db_name)

In [12]:
if con is not None:
    t_data = pd.read_sql("""SELECT Title FROM no_dup_train""",con)
else:
    print("Connection Error!!")

In [13]:
titile_data.shape

(4206308, 1)

In [21]:
if con is not None:
    tag_data = pd.read_sql("""SELECT Tags FROM no_dup_train""",con)
else:
    print("Connection Error!!")

In [22]:
tags_data.shape

(4206308, 1)

In [16]:
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [17]:
import re

#t_data.head()
t_data.Title = t_data.Title.apply(lambda x : x.encode('utf-8'))
t_data.Title = t_data.Title.apply(lambda x : str.lower(x))
t_data.Title = t_data.Title.apply(lambda x : re.sub(r'[^A-Za-z0-9#+.\-]+',' ',x))
#title_data = t_data

In [18]:
def fn(sen):
    return ' '.join(w for w in word_tokenize(sen) if w not in stop_words)

t_data.Title = t_data.Title.apply(lambda x : fn(x))#' '.join for w in word_tokenize(x) if w not in stop_words)

In [27]:
title_data = t_data

## --------------------

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

In [30]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')  #Binary BoW
multi_lab_y = vectorizer.fit_transform(tag_data.Tags)

In [31]:
print("Number of data points :", multi_lab_y.shape[0])
print("Number of unique tags :", multi_lab_y.shape[1])

('Number of data points :', 4206308)
('Number of unique tags :', 42048)


## Test Train Split

In [33]:
tot_size = title_data.shape[0]
train_size = int(tot_size * 0.8)
print(tot_size,train_size)

(4206308, 3365046)


In [34]:
x_train=title_data.head(train_size)
x_test=title_data.tail(tot_size - train_size)

y_train = multi_lab_y[0:train_size,:]
y_test = multi_lab_y[train_size:tot_size,:]

## @Model Testing

In [35]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import LinearSVC

from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score

In [36]:
vectorizer = TfidfVectorizer(min_df=0.00009, max_features=200000, smooth_idf=True, norm="l2", \
                             tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(1,3))

In [38]:
start = datetime.now()
x_train_multilabel = vectorizer.fit_transform(x_train.Title)     ## replace column name appropiately
x_test_multilabel = vectorizer.transform(x_test.Title)
print("Time taken to run this cell :", datetime.now() - start)

('Time taken to run this cell :', datetime.timedelta(0, 265, 49696))


In [39]:
print("Dimensions of train data X:",x_train_multilabel.shape, "Y :",y_train.shape)
print("Dimensions of test data X:",x_test_multilabel.shape,"Y:",y_test.shape)

('Dimensions of train data X:', (3365046, 8398), 'Y :', (3365046, 42048))
('Dimensions of test data X:', (841262, 8398), 'Y:', (841262, 42048))


## SGDC with OnevsRest

In [40]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.00001, penalty='l1'), n_jobs=-1)
classifier.fit(x_train_multilabel, y_train)
predictions = classifier.predict(x_test_multilabel)

print("Time taken to run this cell :", datetime.now() - start)
print("accuracy :",metrics.accuracy_score(y_test,predictions))
#print("macro f1 score :",metrics.f1_score(y_test, predictions, average = 'macro'))
print("micro f1 scoore :",metrics.f1_score(y_test, predictions, average = 'micro'))
#print("hamming loss :",metrics.hamming_loss(y_test,predictions))
#print("Precision recall report :\n",metrics.classification_report(y_test, predictions))


IOError: [Errno 28] No space left on device